In [1]:
# 导入需要的包
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import random

第1题 代码阅读与任务解读，过程如下

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for label, sub_dir in enumerate(os.listdir(root_dir)):
            print(label)  #只需要把label都打出来就可以看出来这段代码是什么意思
            full_dir = os.path.join(root_dir, sub_dir)
            for img_name in os.listdir(full_dir):
                img_path = os.path.join(full_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


# 数据预处理
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 创建数据加载器
# 真实数据label为0，生成图像label为1
train_dir = 'cifar_uvit/train'
test_dir = 'cifar_uvit/test'

train_dataset = CustomDataset(train_dir, transform=transform)
test_dataset = CustomDataset(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

0
1
0
1


第2、3题 请将小爱同学的代码补全，并执行训练和测试过程

In [3]:
class FixNet(nn.Module):
    def __init__(self):
        super(FixNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6 * 15 * 15, 84)
        self.fc2 = nn.Linear(84, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 6 * 15 * 15)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.activation(x)
        return x    

In [4]:
# 训练函数
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    max_accuracy = 0 # 打印最高准确率
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        test_accuracy = eval_model(model, test_loader)
        train_accuracy = eval_model(model,train_loader)
        if test_accuracy > max_accuracy:
            max_accuracy = test_accuracy
        
        log_message = f'Epoch {epoch+1}/{num_epochs}, Xiao Ai Train Accuracy: {train_accuracy:.4f},Xiao Ai Test Accuracy: {test_accuracy:.4f}\n'
        print(log_message)
    #print("max_accuracy:", max_accuracy)

# 评估函数
def eval_model(model, test_loader):
    model.eval()
    corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            preds = outputs >= 0.5
            corrects += torch.sum(preds == labels).item()
            total += labels.size(0)
    accuracy = corrects / total
    return accuracy

In [5]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 创建模型、损失函数和优化器
model = FixNet().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型并记录日志
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=5)

Using device: cpu


C:\ProgramData\Anaconda3\envs\ai4engineeringcourse\lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Epoch 1/5, Xiao Ai Train Accuracy: 0.6754,Xiao Ai Test Accuracy: 0.6775

Epoch 2/5, Xiao Ai Train Accuracy: 0.6675,Xiao Ai Test Accuracy: 0.6640

Epoch 3/5, Xiao Ai Train Accuracy: 0.7199,Xiao Ai Test Accuracy: 0.7190

Epoch 4/5, Xiao Ai Train Accuracy: 0.7335,Xiao Ai Test Accuracy: 0.7240

Epoch 5/5, Xiao Ai Train Accuracy: 0.7430,Xiao Ai Test Accuracy: 0.7255



第4题 请搭建如下图所示的LeNet-5模型，实现训练和测试过程，过程如下：

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [7]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 创建模型、损失函数和优化器
model = LeNet().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型并记录日志
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=5)

Using device: cpu
Epoch 1/5, Xiao Ai Train Accuracy: 0.6573,Xiao Ai Test Accuracy: 0.6580

Epoch 2/5, Xiao Ai Train Accuracy: 0.7225,Xiao Ai Test Accuracy: 0.7255

Epoch 3/5, Xiao Ai Train Accuracy: 0.7224,Xiao Ai Test Accuracy: 0.7305

Epoch 4/5, Xiao Ai Train Accuracy: 0.7374,Xiao Ai Test Accuracy: 0.7315

Epoch 5/5, Xiao Ai Train Accuracy: 0.7362,Xiao Ai Test Accuracy: 0.7100



第5题 请设计一个与LeNet-5结构不同的卷积神经网络，实现和测试过程，过程如下：

In [8]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 84)
        self.fc2 = nn.Linear(84, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [9]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 创建模型、损失函数和优化器
model = MyNet().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型并记录日志
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=5)

Using device: cpu
Epoch 1/5, Xiao Ai Train Accuracy: 0.7188,Xiao Ai Test Accuracy: 0.7300

Epoch 2/5, Xiao Ai Train Accuracy: 0.7192,Xiao Ai Test Accuracy: 0.7295

Epoch 3/5, Xiao Ai Train Accuracy: 0.7281,Xiao Ai Test Accuracy: 0.7440

Epoch 4/5, Xiao Ai Train Accuracy: 0.7349,Xiao Ai Test Accuracy: 0.7230

Epoch 5/5, Xiao Ai Train Accuracy: 0.7493,Xiao Ai Test Accuracy: 0.7365

